In [1]:
import os
import glob
import shutil

import autotst.reaction
import arkane.ess.gaussian
import rmgpy.chemkin

# from hotbit import Hotbit

stats.py:37 <module> WARNING Optional package dependency "xlwt" not loaded. Some output features will not work.


In [2]:
import sys


sys.path.append('/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts/thermo/')
sys.path.append('/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts/kinetics/')

import job
import importlib
importlib.reload(job)
import kineticfun

In [3]:
# reaction_index = 288
# reaction_index = 805
# reaction_index = 748
reaction_index = 720
reaction_index = 722
reaction_smiles = kineticfun.reaction_index2smiles(reaction_index)

In [4]:
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft"
reaction_dir = os.path.join(DFT_DIR, 'kinetics', f'reaction_{reaction_index:04}')
overall_dir = os.path.join(reaction_dir, 'overall')
arkane_dir = os.path.join(reaction_dir, 'arkane')
os.makedirs(arkane_dir, exist_ok=True)

species_dict_file = "/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model/species_dictionary.txt"
species_dict = rmgpy.chemkin.load_species_dictionary(species_dict_file)


In [5]:
direction = 'forward'
reaction = autotst.reaction.Reaction(label=reaction_smiles)
reaction.ts[direction][0].get_molecules()
# reaction.generate_conformers(ase_calculator=Hotbit())



reaction.py:168 load_databases INFO Loading RMG database from '/home/harris.se/rmg/RMG-database/input'
transport.py:294 load_groups INFO Loading transport group database from /home/harris.se/rmg/RMG-database/input/transport/groups...
statmech.py:541 load_libraries INFO Loading frequencies library from halogens_G4.py in /home/harris.se/rmg/RMG-database/input/statmech/libraries...
statmech.py:555 load_groups INFO Loading frequencies group database from /home/harris.se/rmg/RMG-database/input/statmech/groups...
thermo.py:945 load_libraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /home/harris.se/rmg/RMG-database/input/thermo/libraries...
thermo.py:945 load_libraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /home/harris.se/rmg/RMG-database/input/thermo/libraries...
thermo.py:945 load_libraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /home/harris.se/rmg/RMG-database/input/thermo/libraries...
thermo.py:976 load_group

reaction.py:359 get_labeled_reaction ERROR Couldn't match <Molecule "C=C[CH]C"> + <Molecule "C=C[O]"> <=> <Molecule "C=C=O"> + <Molecule "C=CCC"> to intra_H_migration, trying different combination...
reaction.py:359 get_labeled_reaction ERROR Couldn't match <Molecule "[CH2]C=CC"> + <Molecule "[CH2]C=O"> <=> <Molecule "C=C=O"> + <Molecule "C=CCC"> to intra_H_migration, trying different combination...
reaction.py:359 get_labeled_reaction ERROR Couldn't match <Molecule "[CH2]C=CC"> + <Molecule "C=C[O]"> <=> <Molecule "C=C=O"> + <Molecule "C=CCC"> to intra_H_migration, trying different combination...
reaction.py:262 generate_distance_data INFO Distance between *1 and *3 is too small, setting it to lower bound of uncertainty
reaction.py:268 generate_distance_data INFO The distance data is as follows: DistanceData(distances={'d12': 1.442920,'d13': 2.689288,'d23': 1.251105,}, uncertainties={'d12': 0.000809,'d13': 0.000463,'d23': 0.000067,})
reaction.py:771 get_rdkit_mol INFO Initially embedde

(<rdkit.Chem.rdchem.Mol at 0x2b2f2830d710>, Atoms(symbols='OC6H10', pbc=False))

In [7]:
# pick the lowest energy transition state:
TS_logs = glob.glob(os.path.join(overall_dir, f'fwd_ts_*.log'))
TS_log = ''
lowest_energy = 0
for logfile in TS_logs:
    g_reader = arkane.ess.gaussian.GaussianLog(logfile)
    energy = g_reader.load_energy()
    if energy < lowest_energy:
        lowest_energy = energy
        TS_log = logfile

LogError: There was an error (Unconverged) with Gaussian output file /work/westgroup/harris.se/autoscience/autoscience/butane/dft/kinetics/reaction_0722/overall/fwd_ts_0000.log due to line:
 Error termination via Lnk1e in /shared/centos7/gaussian/g16/l9999.exe at Thu Oct 13 21:36:40 2022.


In [ ]:
def get_sp_name(smiles):
    if smiles == '[CH2]C=CC':
        smiles = 'C=C[CH]C'
    for entry in species_dict.keys():
        if species_dict[entry].smiles == smiles:
            return str(species_dict[entry])

In [ ]:
def get_reaction_label(rmg_reaction):
    label = ''
    for reactant in rmg_reaction.reactants:
        label += get_sp_name(reactant.smiles) + ' + '
    label = label[:-2]
    label += '<=> '
    for product in rmg_reaction.products:
        label += get_sp_name(product.smiles) + ' + '
    label = label[:-3]
    return label

In [ ]:
# write the input file

model_chemistry = 'M06-2X/cc-pVTZ'

lines = [
    f'modelChemistry = "{model_chemistry}"\n',
    'useHinderedRotors = False\n',
    'useBondCorrections = False\n\n',
]

for reactant in reaction.rmg_reaction.reactants + reaction.rmg_reaction.products:
    species_smiles = reactant.smiles
    print(species_smiles)
    if species_smiles == '[CH2]C=CC':
        species_smiles = 'C=C[CH]C'
    
    species_name = get_sp_name(species_smiles)
    species_index = job.smiles2index(species_smiles)
    species_arkane_dir = os.path.join(DFT_DIR, 'thermo', f'species_{species_index:04}', 'arkane')
    
    species_file = os.path.join(f'species_{species_index:04}', os.path.basename(glob.glob(os.path.join(species_arkane_dir, 'conformer_*.py'))[0]))

    try:
        shutil.copytree(species_arkane_dir, os.path.join(arkane_dir, f'species_{species_index:04}'))
    except FileExistsError:
        pass
        
    # TODO - copy the species into the destination so the arkane calculation can be copied and redone elsewhere
    lines.append(f'species("{species_name}", "{species_file}", structure=SMILES("{species_smiles}"))\n')
    lines.append(f'thermo("{species_name}", "NASA")\n\n')


lines.append('\n')


TS_name = f'TS'
TS_file = f'TS.py'
TS_arkane_path = os.path.join(arkane_dir, TS_file)
shutil.copy(TS_log, arkane_dir)

lines.append(f'transitionState("{TS_name}", "{TS_file}")\n')

reaction_label = get_reaction_label(reaction.rmg_reaction)
reactants = [get_sp_name(reactant.smiles) for reactant in reaction.rmg_reaction.reactants]
products = [get_sp_name(product.smiles) for product in reaction.rmg_reaction.products]
lines.append(f'reaction(\n')
lines.append(f'    label = "{reaction_label}",\n')
lines.append(f'    reactants = {reactants},\n')
lines.append(f'    products = {products},\n')
lines.append(f'    transitionState = "{TS_name}",\n')
lines.append(f'    tunneling = "Eckart",\n')
lines.append(f')\n\n')


lines.append(f'statmech("{TS_name}")\n')
lines.append(f'kinetics("{reaction_label}")\n\n')

# write the TS file
ts_lines = [
    'energy = {"' + f'{model_chemistry}": Log("{os.path.basename(TS_log)}")' + '}\n\n',
    'geometry = Log("' + f'{os.path.basename(TS_log)}")' + '\n\n',
    'frequencies = Log("' + f'{os.path.basename(TS_log)}")' + '\n\n',
]
with open(TS_arkane_path, 'w') as g:
    g.writelines(ts_lines)


arkane_input_file = os.path.join(arkane_dir, 'input.py')
with open(arkane_input_file, 'w') as f:
    f.writelines(lines)


In [ ]:
# make the slurm script to run arkane
run_script = os.path.join(arkane_dir, 'run_arkane.sh')
with open(run_script, 'w') as f:
    # Run on express
    f.write('#!/bin/bash\n')
    f.write('#SBATCH --partition=express,short,west\n')
    f.write('#SBATCH --time=00:20:00\n\n')
    f.write('python ~/rmg/RMG-Py/Arkane.py input.py\n\n')
